In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.time import Time
from astropy import constants as const
import astropy.coordinates as coord
from astropy.table import Table

In [3]:
# setting some plotting style stuff for my taste
import matplotlib
matplotlib.rcParams.update({'font.size':18})
matplotlib.rcParams.update({'font.family':'serif'})

In [5]:
# lists of systems to explore (exoplanets here), what we really want is (ra,dec,dist) from these

# planetary systems table:
# https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=PS&constraint=default_flag=1
Pfile = 'PS_2021.04.19_17.09.05.csv'
df = pd.read_csv(Pfile, comment='#')

# TESS project candidates
# https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=TOI
Tfile = 'TOI_2021.04.19_17.09.56.csv'
dfT = pd.read_csv(Tfile, comment='#')

# clean these lists...
# use Pandas GroupBy & First to select only unique Gaia IDs (the first entries)
stars1 = df.groupby('gaia_id',as_index=False).first()
stars2 = dfT.groupby('tid',as_index=False).first()

### workflow shakedown

1. Prep Sources (Novae)
- read in list(s)
- put into same structures for matching

2. Prep the Stars:
- put ra,dec,dist into 3D SkyCoord object w/ units
- Time into astropy.Time object w/ units


3. Compute the Ellipse Positions
- full 3D distances, thank you astropy for doing this 
- then dsitances math from before... need to name this step better



In [6]:
# 1.
def make_objects(ra,dec,dist, Y, M, D):
    cc = SkyCoord(ra=ra, dec=dec, distance=dist)

    tt = Time({'year':Y, 'month':M, 'day':D}, 
              format='ymdhms')

    return cc,tt

def pull_sources(file='SOURCES_20210419.csv'):
    '''
    A simple wrapper to pull the "sources", aka the novae/SNe/original
    events as WE observed them, and put them into astropy structures.
    
    TO DO: sure wish we had errors, esp. on distance...
    '''
    src = pd.read_csv(file)

    cc, tt = make_objects(src['RA'].values*u.deg, 
                          src['Dec'].values*u.deg, 
                          src['dist'].values*u.pc, 
                          src['year'].astype('int').values, 
                          src['month'].astype('int').values, 
                          src['day'].astype('int').values)
    return cc, tt

cc_S, tt_S = pull_sources() # the event "sources"

/Users/james/opt/anaconda3/lib/python3.8/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 105 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [7]:
# 2.
Sok1 = np.where(np.isfinite(stars1['sy_dist']))[0]
Sok2 = np.where(np.isfinite(stars2['st_dist']))[0]

# Coordinates for the stars/objects to monitor
cc_O = SkyCoord(ra=np.concatenate((stars1['ra'][Sok1].values, 
                                   stars2['ra'][Sok2].values)) * u.deg, 
                dec=np.concatenate((stars1['dec'][Sok1].values,
                                    stars2['dec'][Sok2].values)) * u.deg, 
                distance=np.concatenate((stars1['sy_dist'][Sok1].values,
                                         stars2['st_dist'][Sok2].values)) * u.pc)


In [8]:
# 3.

etol = 1. # ellipse tolerance (in lyr) (1 year for now... )
print('yr', '<0.07lyr', '<0.1lyr', '<1lyr')

# try a few years, just for this example
for j in (range(2019,2023)):
    # create time object for year "j", set to Jan 1
    tt_0 = Time( {'year':np.zeros(len(cc_O), dtype=int) + j, 
                  'month':np.zeros(len(cc_O), dtype=int) + 1, 
                  'day':np.zeros(len(cc_O), dtype=int) + 1}, format='ymdhms')

    # the smaller list is the novae/sources... 
    # loop over each Source, compute the ellipsoid distance to each star
    for k in range(tt_S.size):
        # time diff from Source[k] to all Stars/Objects
        # should be positive, for Novae in the past
        dt = tt_0 - tt_S[k]

        c = cc_S[k].distance.to('lyr') / 2 # the Source[k] foci distance
        a = (((dt.to('s') * const.c) / 2) + c).to('lyr') # the semi-major axis of Source[k] ellipse
        d2 = cc_O.separation_3d(cc_S[k]) # 3D dist between Source[k] and the Stars/Objects

        # the ellipsoid surface distance
        Edist = (cc_O.distance.to('lyr').value + d2.to('lyr').value) - (2 * a.to('lyr').value)

        # we should LOOK at sources if they are near the ellipsoid edges
        # (and if they have positive dt)
        look = (np.abs(Edist) < etol)  & (dt > 0)

        # print the SN1987A results for each year
        if (tt_S[k].value[0] == 1987) & (tt_S[k].value[1] == 2):
            print('SN1987A, <0.07lyr: ', sum((np.abs(Edist) < 0.1)  & (dt > 0)))
            
    print(j, sum(look))
    print('')

yr <0.07lyr <0.1lyr <1lyr


/Users/james/opt/anaconda3/lib/python3.8/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/Users/james/opt/anaconda3/lib/python3.8/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


SN1987A, <0.07lyr:  2
2019 21

SN1987A, <0.07lyr:  1
2020 20

SN1987A, <0.07lyr:  0
2021 17

SN1987A, <0.07lyr:  1
2022 15

